## Criando um dataset para treino

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_inicio = "2025-01-01 00:00:00"
data_fim = "2025-06-01 00:00:00"
frequencia = "15T" # 15 minutos

print("Gerando dados sintéticos...")
datas = pd.date_range(start=data_inicio, end=data_fim, freq=frequencia)
df = pd.DataFrame(index=datas)

df['vazao'] = 0.2
perfil_diario = {
    0:0.1, 1:0.1, 2:0.1, 3:0.1, 4:0.2, 5:1.0,
    6:8.0, 7:15.0, 8:10.0, 9:5.0, 10:4.0, 11:7.0,
    12:12.0, 13:10.0, 14:4.0, 15:3.0, 16:3.0, 17:5.0,
    18:10.0, 19:18.0, 20:15.0, 21:8.0, 22:4.0, 23:1.0
}
perfil_semanal = {0:1.0, 1:1.0, 2:1.05, 3:1.0, 4:0.95, 5:1.2, 6:1.15}

horas = df.index.hour
dias_semana = df.index.dayofweek

df['vazao'] += (horas.map(perfil_diario) * dias_semana.map(perfil_semanal))
df['vazao'] += np.random.normal(0, 0.5, len(df)) # Ruído
df['vazao'] = df['vazao'].clip(lower=0) # Sem vazão negativa

df['volume_intervalo'] = df['vazao'] * 15.0


df['volume_acumulado_dia'] = df.groupby(df.index.date)['volume_intervalo'].cumsum()


df = df.dropna()

df.to_csv('dados_vazao.csv')
print("Dataset gerado: 'dados_vazao_com_volume.csv'")
print(df[['vazao', 'volume_acumulado_dia']].head())

Gerando dados sintéticos...


/tmp/ipython-input-562123432.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  datas = pd.date_range(start=data_inicio, end=data_fim, freq=frequencia)


Dataset gerado: 'dados_vazao_com_volume.csv'
                        vazao  volume_acumulado_dia
2025-01-01 00:00:00  0.276251              4.143759
2025-01-01 00:15:00  0.165789              6.630588
2025-01-01 00:30:00  0.251466             10.402577
2025-01-01 00:45:00  0.268439             14.429156
2025-01-01 01:00:00  0.397130             20.386107


## Treinando o modelo de rede neural

Importando as bibliotecas

In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


Extraindo os dados e fazendo a divisão entre treino e teste

In [22]:
df = pd.read_csv('/content/dados_vazao.csv', index_col=0, parse_dates=True)

df['hora'] = df.index.hour
df['dia_semana'] = df.index.dayofweek

features = ['hora', 'dia_semana', 'vazao', 'volume_acumulado_dia']
target = 'vazao'

X = df[features].values
y = df[target].values

## Fazendo a normalização dos dados

In [23]:
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))


Copiando dados para o C++

In [24]:
print("\n--- COPIE ISTO PARA O C++ ---")
print(f"const float X_min[4] = {{ {', '.join([f'{x:.6f}' for x in scaler_X.data_min_])} }};")
print(f"const float X_scale[4] = {{ {', '.join([f'{x:.6f}' for x in scaler_X.scale_])} }};")
print(f"const float y_min = {scaler_y.data_min_[0]:.6f};")
print(f"const float y_scale = {scaler_y.scale_[0]:.6f};")
print("-----------------------------\n")



--- COPIE ISTO PARA O C++ ---
const float X_min[4] = { 0.000000, 0.000000, 0.000000, 0.000000 };
const float X_scale[4] = { 0.043478, 0.166667, 0.043168, 0.000092 };
const float y_min = 0.000000;
const float y_scale = 0.043168;
-----------------------------



Dividindo dados entre treion e teste

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

Treinando o modelo de rede neural

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=[4]), # 4 Entradas
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1) # 1 Saída (Previsão)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0184 - mae: 0.0893
Epoch 2/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.3803e-04 - mae: 0.0117
Epoch 3/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.3966e-05 - mae: 0.0053
Epoch 4/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.7867e-05 - mae: 0.0031
Epoch 5/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7.6027e-06 - mae: 0.0020
Epoch 6/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.9348e-06 - mae: 0.0014
Epoch 7/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.3969e-06 - mae: 0.0010
Epoch 8/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.6437e-06 - mae: 7.9487e-04
Epoch 9/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.4556e-06 - mae: 7.6008e-04
Epoch 10/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0802e-06 - mae: 6.3727e-04
Epoch 11/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1183e-06 - mae: 7.1042e-04
Epoch 12/50
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1

Mensurando valor do Erro Médio Absoluto

In [27]:
loss, mae = model.evaluate(X_test, y_test)
print(f"\nErro Médio Absoluto (Normalizado): {mae:.4f}")

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1051e-07 - mae: 3.1035e-04

Erro Médio Absoluto (Normalizado): 0.0003


Convertendo o modelo para o ESP32

In [28]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('modelo_vazao.tflite', 'wb') as f:
    f.write(tflite_model)

print("Modelo salvo: 'modelo_vazao.tflite'")
print("Não esqueça de rodar: xxd -i modelo_vazao.tflite > modelo_vazao.h")

Saved artifact at '/tmp/tmpe_5h1rnx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136379857513808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379857512464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379857513424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379857502480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379857512272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136379857502672: TensorSpec(shape=(), dtype=tf.resource, name=None)
Modelo salvo: 'modelo_vazao.tflite'
Não esqueça de rodar: xxd -i modelo_vazao.tflite > modelo_vazao.h
